# LCEL Chain

In [2]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, PromptTemplate, ChatPromptTemplate

base_url = "http://localhost:11434"
model = "phi3"

llm = ChatOllama(base_url=base_url,
                 model=model,
                 temperature=0.5,
                 num_predict=256
                 )

In [23]:
system = SystemMessagePromptTemplate.from_template('you are a {level} guitar teacher! you answer in short sentences')
question = HumanMessagePromptTemplate.from_template('tell me about {topics} in {points} points')

In [24]:
messages = [system, question]
template = ChatPromptTemplate(messages)

In [25]:
chain = template | llm

In [26]:
response = chain.invoke({'level':'advanced', 'topics': 'penta-blues', 'points':3})

In [9]:
print(response.content)

1. Penta-blues is not an established genre; the term seems to be either a misnomer or a niche style combining elements of blues with pentatonic scales, possibly used for educational purposes like "Pentatonica."

2. If it were meant as a teaching tool in guitar education, penta-blues would likely involve simplifying traditional 12-bar blues progressions using only the notes from the minor pentatonic scale to help beginners understand and play basic blues music comfortably on their instrument.

3. As an educational concept within "Pentatonica," this hypothetical style might focus on cultural aspects of African American blues, teaching students about its historical significance while using a more accessible approach with fewer chords for ease of learning.


## Analysis of the previous output

 - Adding StrOutputParser in output of your chain

In [1]:
from langchain_core.output_parsers import StrOutputParser

In [ ]:
analysis_prompt = ChatPromptTemplate.from_template("""analyse the following {response} verify and tell me how dificult it is to understand.
                                     Answer in one sentence only.
                                     """)

answer_analysis = analysis_prompt | llm | StrOutputParser()
output = answer_analysis.invoke({'response': response})
print(output)

Understanding penta-blues requires a grasp of music theory and guitar techniques but isn't overly complex for those with musical background or interest.


## Composed chain

 - Multiple chains runnables

In [30]:
composed_chain = {'response': chain} | analysis_prompt | llm | StrOutputParser()

In [31]:
output = composed_chain.invoke({'level':'advanced', 'topics': 'fusion improvisation', 'points':3})

In [32]:
print(output)

Understanding the provided content requires some familiarity with music theory but isn't overly difficult for someone interested in or knowledgeable about modern musical genres.


# Parallel Chains

In [8]:
system = SystemMessagePromptTemplate.from_template('you are a {level} guitar teacher! you answer in short sentences')
question = HumanMessagePromptTemplate.from_template('tell me about {topics} in {points} points')

messages = [system, question]
template = ChatPromptTemplate(messages)

teacher_chain = template | llm | StrOutputParser()

response = teacher_chain.invoke({'level':'advanced', 'topics': 'polyrhythm left hand', 'points':2})

print(response)

1. In jazz, the left hand often plays complex rhythms like polyrhythms to create syncopation and groove; for example, a bassist might play triplets against duplets on the snare drum.

2. Mastering these patterns requires understanding subdivisions within beats and practicing slowly with metronome or backing tracks before attempting live performance.


In [9]:
question = HumanMessagePromptTemplate.from_template('write a rime of {topics} in {sentences} lines')

messages = [system, question]
template = ChatPromptTemplate(messages)

rime_chain = template | llm | StrOutputParser()

response = rime_chain.invoke({'level':'advanced', 'topics': 'polyrhythm left hand', 'sentences':2})

print(response)

In the realm where rhythms intertwine,  
Left fingers dance and design.  
Drums beat with two pulses at once,  
A syncopated harmony that bonds.


In [12]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(teacher=teacher_chain, rime=rime_chain)

output = chain.invoke({'level':'advanced', 'topics': 'polyrhythm left hand', 'sentences':1, 'points': 2})

print(output.get('teacher'))
print(output.get('rime'))

1. Polyrhythms involve multiple overlapping rhythmic patterns played simultaneously, and they can be challenging for the left hand on a guitar because it requires precise timing to play different notes against each beat of those patterns. This often involves shifting between chords or strings that align with specific beats in one pattern while playing another concurrently.

2. To effectively practice polyrhythms, start by clapping and counting the rhythms separately before transferring them onto your guitar fretboard using open string notes. Gradually increase complexity as you become more comfortable maintaining accuracy on each hand independently yet together in harmony.
"In the realm where rhythms intertwine,  
A pattern emerges with design so divine.
Four beats cascade as if they were one;   
Triplet steps beneath them shone."


## Chain Router

In [35]:
prompt = """
Given the user review below, classify it as either being about 'Positive' or 'Negative'
Do not respond with more than one word.

review: {review}
classification:
"""

In [36]:
template = ChatPromptTemplate.from_template(prompt)

In [37]:
chain = template | llm | StrOutputParser()

In [38]:
review = "Thank you so much to providing such a great platform for learning, I'm really happy with your service"
response = chain.invoke(review)
print(response)


Positive


In [29]:
review = "você é um saco de merda, sua plataforma fede e você é um escroto"
response = chain.invoke(review)
print(response)


Negative


## Instruction 2 (More Diffrances) - Increased difficulty with additional constraints.

Given the detailed narrative below about a user's experience on an online forum, identify and extract direct quotes that can be used to illustrate instances of 'Positive Feedback', 'Negative Feedback', or 'Neutral Commentary'. Ensure your extractions account for sarcasm by interpreting the tone in context. Additionally, categorize each extracted quote under its corresponding sentiment label ('Positive Feedback,' 'Negative Feedback,' or 'Neutral Commentary'). Note that a single sentence may contain multiple sentiments and should be classified accordingly based on predominant sentiment within it:

review_text = "I've been using this forum for years, and I must say the community spirit here is unmatched. However, yesterday was frustrating when my questions went ignored; they simply disappeared into thin air! On a brighter note, their customer service reached out to me quickly afterward."

e

In [47]:
positive_prompt = """
You are an expert in writing reply for positive reviews.
You need to encourage the user to share their experience on social media.

review: {review}
Answer:
"""

positive_template = ChatPromptTemplate.from_template(positive_prompt)
positive_chain = positive_template | llm | StrOutputParser()

In [48]:
negative_prompt = """
You are an expert in writing reply for negatives reviews.
You need first to apologize for the incovenience caused to the user.
You need to encourage the user to share their concern on following email: support@business.com

review: {review}
Answer:
"""

negative_template = ChatPromptTemplate.from_template(negative_prompt)
negative_chain = negative_template | llm | StrOutputParser()

In [53]:
def route(info):
    print(info)
    if info['sentiment'].lower().endswith('positive'):
        return positive_chain
    else:
        return negative_chain

In [54]:
from langchain_core.runnables import RunnableLambda

full_chain = {'sentiment': chain, 'review': lambda x: x['review']} | RunnableLambda(route)

full_chain

{
  sentiment: ChatPromptTemplate(input_variables=['review'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['review'], input_types={}, partial_variables={}, template="\nGiven the user review below, classify it as either being about 'Positive' or 'Negative'\nDo not respond with more than one word.\n\nreview: {review}\nclassification:\n"), additional_kwargs={})])
             | ChatOllama(model='phi3', num_predict=256, temperature=0.5, base_url='http://localhost:11434')
             | StrOutputParser(),
  review: RunnableLambda(lambda x: x['review'])
}
| RunnableLambda(route)

In [60]:
review = 'your service small like shit'
review

'your service small like shit'

In [61]:
output = full_chain.invoke({'review':review})

{'sentiment': '\nNegative', 'review': 'your service small like shit'}


In [62]:
import textwrap
print(textwrap.fill(output))


 I'm truly sorry to hear that you had a less than satisfactory
experience with our services, and I sincerely apologize for any
inconvenience this may have caused you. Your feedback is incredibly
valuable to us, as it helps identify areas where we can improve. If
there are specific concerns or issues you would like to discuss
further, please feel free to reach out directly via email at
support@business.com so that our team can address them promptly and
effectively. We genuinely appreciate your candid review and look
forward to the opportunity of serving you better in the future.


# Custom Chains Runnables with RunnablePassthrough and RunnableLambda

In [67]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

In [68]:
def char_count(text):
    return len(text)

def word_count(text):
    return len(text.split())

In [ ]:
prompt = ChatPromptTemplate.from_template("""Explain there inputs {input1} and {input2}""")

chain = prompt | llm | StrOutputParser() | {'char_count': RunnableLambda(char_count),
                                            'word_count': RunnableLambda(word_count),
                                            'output': RunnablePassthrough()}
output = chain.invoke({'input1': 'what are a raytracer?', 'input2': 'what are a pathtracer?'})
print(output.get('char_count'))
print(output.get('word_count'))
print(textwrap.fill(output.get('output')))

1192
184
A Ray Tracer: In computer graphics, a ray tracing is an algorithm that
simulthemethod of producing high-quality images by modelling the
behaviorof light as it interacts with objects in a scene. It traces
every single path (ray) that visible light could take from any given
point on or near the object to the viewer's eye, taking into account
reflection and refraction at each surface along its way until all of
these rays have been traced backward through time—from their final
destination in the camera sensor. Ray Tracers are used extensively for
producing photorealistic images by simulating complex lighting
phenomena like shadows, reflection, refraction or even caustics
(focused patterns on a surface due to reflected and transmitted
waves). The major advantage of ray tracing is its ability to produce
highly realistic rendering results. However, it has high computational
cost which makes the process slow in comparison with other techniques
such as rasterization.  A Path Tracer:  P

# Custom Chains

In [75]:
from langchain_core.runnables import chain

In [76]:
@chain
def custom_chain(params):
    return {
        "teacher": teacher_chain.invoke(params),
        "rime": rime_chain.invoke(params),
    }

In [77]:
params = {'level':'advanced', 'topics': 'polyrhythm left hand', 'sentences':1, 'points': 2}

output = custom_chain.invoke(params)

print(output.get('teacher'))
print(output.get('rime'))

1. Polyrhythms involve using multiple conflicting rhythmic patterns simultaneously, often requiring different parts of the hands to play contrasting beats; for example, playing a triplet pattern with one finger while accentuating off-beats or eighth notes on another hand can create polyrhythmic effects.
2. Mastery of left hand independence is essential in executing complex and syncopated rhythms as it allows guitarists to maintain steady metronome patterns (like quarter notes) with one part while playing intricate, time-shifting melodies or chords on another string simultaneously using the other fingers.
As my fingers dance on strings, I play with two beats at once; the bass and snare align like stars against night's dark. In this intricate pattern where time divides into fractions four parts make—a quadruple rhythm unfolds before your eyes to take!
